In [2]:
import pandas as pd
import numpy as np
from random import sample
from random import seed
from scipy.stats import ttest_ind
from Ames_Functions import *
pd.set_option("display.max_columns", None)

In [3]:
df_features_all = load_state_pkl()['df_house_both_after_feature_engineering'][0]
saleprice_2019 = load_state_pkl()["df_RE_latlong2019"][0].SalePrice
saleprice_2021 = load_state_pkl()["df_RE_latlong2021"][0].SalePrice

## Hypothesis testing numeric fields using ttest

In [4]:
def ttest_2019_2021 (field):
    return ttest_ind(df_features_all.query('Year == 2019')[field], df_features_all.query('Year == 2021')[field])

In [5]:
df_features_all

,Rcrd_Yr,InstOne_Yr,TtlVal_AsrYr,LotArea,YrBuilt,TtlBsmtSF,Fireplaces,PoolArea,GarYrBlt,Cars,GarageArea,Has_Fireplace,Has_Garage,Has_Bsmt,TotalArea,TotalArea_Gar,AsbShng_Ext,AsphShn_Ext,BrkComm_Ext,BrkFace_Ext,CBlock_Ext,CemntBd_Ext,HdBoard_Ext,ImStucc_Ext,MetalSd_Ext,No_Data_Ext,Other_Ext,Plywood_Ext,Stucco_Ext,VinylSd_Ext,Wd_Sdng_Ext,WdShing_Ext,Two_Types_GrType,Attachd_GrType,Basment_GrType,BuiltIn_GrType,Carport_GrType,Detachd_GrType,No_Data_GrType,None_GrType,One_Story_HsStyle,One_Five_Fin_HsStyle,One_Five_Unf_HsStyle,Two_Story_HsStyle,Two_Five_Fin_HsStyle,Two_Five_Unf_HsStyle,No_Data_HsStyle,S_Foyer_HsStyle,S_Level_HsStyle,Closest_Time,Service_Arts_closest,Service_Elem_School_closest,Service_Golf_closest,Service_Gym_closest,Service_High_School_closest,Service_Historic_closest,Service_ISU_closest,Service_Library_closest,Service_Medical_closest,Service_Organic_Groceries_closest,Service_Park_closest,Service_Recreation_closest,Service_Religion_closest,Service_Restaurant_closest,Service_Shopping_closest,Service_Spa_closest,Closest_5,Convenience_secs,Central_historic_feature,biz_180_secs,biz_480_secs,Year,Stone_Ext
SaleID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
92,2019.0,2019.0,300,8746.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0,0,0.0,0,0,0,0,2,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,346.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,381.68,551.2,626.9,0,6,2019,0.0
93,2019.0,2019.0,300,9358.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0,0,0.0,0,0,0,0,2,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,346.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,381.68,551.2,626.9,0,6,2019,0.0
96,2019.0,2019.0,389500,10015.0,2016.0,1142.0,1.0,0.0,2016.0,3.0,647.0,1,1,1,3385.0,4032.0,0,0.0,0,0,0.0,0,0,0,0,0,0.0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,243.1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,339.06,487.6,524.0,0,8,2019,0.0
98,2019.0,2019.0,330800,10280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0,0,0.0,0,0,0,0,2,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,244.3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,340.26,488.8,525.2,0,8,2019,0.0
100,2021.0,2021.0,399800,10840.0,2020.0,1659.0,0.0,0.0,2020.0,3.0,791.0,0,1,1,3318.0,4109.0,0,0.0,0,0,0.0,0,0,0,0,0,0.0,0,0,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,256.8,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,352.76,501.3,537.7,0,8,2019,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22743,2021.0,2021.0,177600,10776.0,1991.0,950.0,0.0,0.0,2001.0,2.0,747.0,0,1,1,1929.0,2676.0,0,0.0,0,0,0.0,0,1,0,0,0,0.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,276.6,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,295.28,402.4,391.4,0,15,2021,0.0
22749,2021.0,2021.0,167300,12395.0,1984.0,864.0,0.0,0.0,1984.0,2.0,484.0,0,1,1,1753.0,2237.0,0,0.0,0,0,0.0,0,1,0,0,0,0.0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,305.2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,323.88,431.0,420.0,0,14,2021,0.0
22756,2021.0,2021.0,162900,10463.0,1991.0,962.0,0.0,0.0,1993.0,1.0,384.0,0,1,1,1963.0,2347.0,0,0.0,0,0,0.0,0,1,0,0,0,0.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,324.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,342.68,449.8,438.8,0,14,2021,0.0


In [6]:
sig_fields = (pd.DataFrame([[n, ttest_2019_2021(n)[1]] for n in ['TotalArea_Gar', 'TtlVal_AsrYr', 'YrBuilt', 'Cars', 'Closest_Time', 'Closest_5', 'Convenience_secs', 'Central_historic_feature', 'biz_180_secs', 'biz_480_secs']],
               columns = ["field", "p_value"])
                .query("p_value < 0.05").field.values
)


In [7]:
display(df_features_all.query("Year == 2019")[sig_fields].describe(), df_features_all.query("Year == 2021")[sig_fields].describe())

,YrBuilt,Cars,Closest_Time,Closest_5,Convenience_secs
count,901.000000,901.000000,901.000000,901.000000,901.000000
mean,1952.964484,1.773896,156.280244,230.535072,370.665927
std,219.663599,0.834944,91.405029,106.177163,122.704920
min,0.000000,0.000000,6.300000,36.920000,145.000000
25%,1955.000000,1.000000,81.500000,137.080000,268.300000
50%,1978.000000,2.000000,125.500000,213.460000,373.400000
75%,2004.000000,2.000000,227.400000,326.120000,462.700000
max,2020.000000,4.000000,392.600000,477.300000,640.200000


,YrBuilt,Cars,Closest_Time,Closest_5,Convenience_secs
count,633.000000,633.000000,633.000000,633.000000,633.000000
mean,1907.685624,1.672780,146.818799,216.309131,356.659558
std,355.199072,0.901528,84.844802,99.695220,118.336055
min,0.000000,0.000000,1.300000,20.880000,134.600000
25%,1947.000000,1.000000,83.500000,136.700000,250.200000
50%,1975.000000,2.000000,118.000000,201.840000,360.600000
75%,2000.000000,2.000000,203.800000,297.360000,447.500000
max,2020.000000,4.000000,371.800000,456.500000,640.200000


In [8]:
df_features_all.query("Year == 2019").query("YrBuilt == 0").shape[0], df_features_all.query("Year == 2021").query("YrBuilt == 0").shape[0]

(11, 21)

In [9]:
ttest_ind(df_features_all.query("YrBuilt != 0 and Year == 2019").YrBuilt, df_features_all.query("YrBuilt != 0 and Year == 2021").YrBuilt)

Ttest_indResult(statistic=2.2761912991955637, pvalue=0.02297497305100568)

In [10]:
df_features_all.query("YrBuilt != 0").groupby('Year').describe()[sig_fields].T.reset_index().query("level_1 == 'mean' or level_1 == 'std'")

Year,level_0,level_1,2019,2021
1,YrBuilt,mean,1977.102247,1973.145425
2,YrBuilt,std,32.751093,33.609582
9,Cars,mean,1.795820,1.730180
10,Cars,std,0.816297,0.860937
17,Closest_Time,mean,154.926966,145.854739
18,Closest_Time,std,90.847469,84.155643
25,Closest_5,mean,228.970449,214.256961
26,Closest_5,std,105.850333,99.766571
33,Convenience_secs,mean,368.854607,352.675327
34,Convenience_secs,std,122.300663,117.972408


## AB Testing

In [11]:
df_features_all
# Has_Fireplace, Has_Garage, Has_Bsmt, Service_Organic_Groceries_closest

,Rcrd_Yr,InstOne_Yr,TtlVal_AsrYr,LotArea,YrBuilt,TtlBsmtSF,Fireplaces,PoolArea,GarYrBlt,Cars,GarageArea,Has_Fireplace,Has_Garage,Has_Bsmt,TotalArea,TotalArea_Gar,AsbShng_Ext,AsphShn_Ext,BrkComm_Ext,BrkFace_Ext,CBlock_Ext,CemntBd_Ext,HdBoard_Ext,ImStucc_Ext,MetalSd_Ext,No_Data_Ext,Other_Ext,Plywood_Ext,Stucco_Ext,VinylSd_Ext,Wd_Sdng_Ext,WdShing_Ext,Two_Types_GrType,Attachd_GrType,Basment_GrType,BuiltIn_GrType,Carport_GrType,Detachd_GrType,No_Data_GrType,None_GrType,One_Story_HsStyle,One_Five_Fin_HsStyle,One_Five_Unf_HsStyle,Two_Story_HsStyle,Two_Five_Fin_HsStyle,Two_Five_Unf_HsStyle,No_Data_HsStyle,S_Foyer_HsStyle,S_Level_HsStyle,Closest_Time,Service_Arts_closest,Service_Elem_School_closest,Service_Golf_closest,Service_Gym_closest,Service_High_School_closest,Service_Historic_closest,Service_ISU_closest,Service_Library_closest,Service_Medical_closest,Service_Organic_Groceries_closest,Service_Park_closest,Service_Recreation_closest,Service_Religion_closest,Service_Restaurant_closest,Service_Shopping_closest,Service_Spa_closest,Closest_5,Convenience_secs,Central_historic_feature,biz_180_secs,biz_480_secs,Year,Stone_Ext
SaleID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
92,2019.0,2019.0,300,8746.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0,0,0.0,0,0,0,0,2,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,346.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,381.68,551.2,626.9,0,6,2019,0.0
93,2019.0,2019.0,300,9358.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0,0,0.0,0,0,0,0,2,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,346.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,381.68,551.2,626.9,0,6,2019,0.0
96,2019.0,2019.0,389500,10015.0,2016.0,1142.0,1.0,0.0,2016.0,3.0,647.0,1,1,1,3385.0,4032.0,0,0.0,0,0,0.0,0,0,0,0,0,0.0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,243.1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,339.06,487.6,524.0,0,8,2019,0.0
98,2019.0,2019.0,330800,10280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0,0,0.0,0,0,0,0,2,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,244.3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,340.26,488.8,525.2,0,8,2019,0.0
100,2021.0,2021.0,399800,10840.0,2020.0,1659.0,0.0,0.0,2020.0,3.0,791.0,0,1,1,3318.0,4109.0,0,0.0,0,0,0.0,0,0,0,0,0,0.0,0,0,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,256.8,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,352.76,501.3,537.7,0,8,2019,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22743,2021.0,2021.0,177600,10776.0,1991.0,950.0,0.0,0.0,2001.0,2.0,747.0,0,1,1,1929.0,2676.0,0,0.0,0,0,0.0,0,1,0,0,0,0.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,276.6,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,295.28,402.4,391.4,0,15,2021,0.0
22749,2021.0,2021.0,167300,12395.0,1984.0,864.0,0.0,0.0,1984.0,2.0,484.0,0,1,1,1753.0,2237.0,0,0.0,0,0,0.0,0,1,0,0,0,0.0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,305.2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,323.88,431.0,420.0,0,14,2021,0.0
22756,2021.0,2021.0,162900,10463.0,1991.0,962.0,0.0,0.0,1993.0,1.0,384.0,0,1,1,1963.0,2347.0,0,0.0,0,0,0.0,0,1,0,0,0,0.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,324.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,342.68,449.8,438.8,0,14,2021,0.0


In [33]:
df_features_all.groupby("Year").Has_Fireplace.value_counts().rename("Counts")[(2019, 0)]

407

In [39]:
total = df_features_all.shape[0]
group2019 = df_features_all.query("Year == 2019").shape[0]
group2021 = df_features_all.query("Year == 2021").shape[0]
group2019_yes = df_features_all.groupby("Year").Has_Fireplace.value_counts().rename("Counts")[(2019, 1)]
group2021_yes = df_features_all.groupby("Year").Has_Fireplace.value_counts().rename("Counts")[(2021, 1)]
total_yes = group2019_yes + group2021_yes
total_no = total - total_yes
group2019_yes_pc = 100 * group2019_yes / group2019
group2021_yes_pc = 100 * group2021_yes / group2021
ab_yes_pc = group2019_yes_pc - group2021_yes_pc

In [40]:
group2021_yes_pc, group2019_yes_pc, ab_yes_pc

(49.13112164296999, 54.827968923418425, 5.696847280448438)

In [44]:
seed(0)
bag1 = [1]*total_yes + [0]*total_no
bag2 = sample(bag1, len(bag1))
group2019_rs = bag2[:group2019]
group2021_rs = bag2[group2019:]
group2019_yes_pc_rs = 100 *sum(group2019_rs)/group2019
group2021_yes_pc_rs = 100 *sum(group2021_rs)/group2021
ab_yes_pc_rs = group2019_yes_pc_rs - group2021_yes_pc_rs

In [45]:
group2021_yes_pc_rs, group2019_yes_pc_rs, ab_yes_pc_rs

(53.23854660347551, 51.94228634850167, -1.296260254973845)